This is a copy of my public kaggle kernel: https://www.kaggle.com/joatom/bqml-population-of-zip-code-per-intersection
# About

This kernel shows how to integrate population and zip code information to the intersections.

The results are writen to the output **pop_zipcode_intersec.csv** if you want to use it in python right away without BigQuery access. 

## Credits
Some of the ideas are inspired by the following kernels. Please visit them and give them upvotes if you like them.
- This kernel is a forked from [BigQuery Machine Learning Tutorial](https://www.kaggle.com/rtatman/bigquery-machine-learning-tutorial).

In [1]:
# Replace 'kaggle-competitions-project' with YOUR OWN project id here --  
PROJECT_ID = 'kaggle-bq-geotag' #
#PROJECT_ID='kaggle-competitions-project'

from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID, location="US")
dataset = client.create_dataset('bqml_example', exists_ok=True)

from google.cloud.bigquery import magics
from kaggle.gcp import KaggleKernelCredentials
magics.context.credentials = KaggleKernelCredentials()
magics.context.project = PROJECT_ID

import seaborn as sns
import matplotlib.pyplot as plt

# create a reference to our table
table = client.get_table("kaggle-competition-datasets.geotab_intersection_congestion.train")

# look at five rows from our dataset
client.list_rows(table, max_results=5).to_dataframe()

,RowId,IntersectionId,Latitude,Longitude,EntryStreetName,ExitStreetName,EntryHeading,ExitHeading,Hour,Weekend,...,TimeFromFirstStop_p40,TimeFromFirstStop_p50,TimeFromFirstStop_p60,TimeFromFirstStop_p80,DistanceToFirstStop_p20,DistanceToFirstStop_p40,DistanceToFirstStop_p50,DistanceToFirstStop_p60,DistanceToFirstStop_p80,City
0,2079854,33,42.34636,-71.09095,Fenway,Fenway,N,N,8,False,...,31,40,44,59,0.0,25.5,32.3,51.8,68.9,Boston
1,2079857,33,42.34636,-71.09095,Fenway,Fenway,N,N,13,False,...,0,10,17,29,0.0,0.0,9.9,21.8,40.4,Boston
2,2079872,33,42.34636,-71.09095,Fenway,Fenway,N,N,15,False,...,22,33,38,60,0.0,43.2,47.3,51.1,65.7,Boston
3,2079893,33,42.34636,-71.09095,Fenway,Fenway,N,N,12,False,...,0,14,18,31,0.0,0.0,15.6,22.5,44.4,Boston
4,2079896,33,42.34636,-71.09095,Fenway,Fenway,N,N,16,False,...,0,23,34,65,0.0,0.0,26.4,52.9,68.9,Boston


In [2]:
%load_ext google.cloud.bigquery

# Loading population and zip code information

Let's have a look at the 2010 population of a zip code area

In [3]:
%%bigquery
SELECT
    SUM(pop.population) AS population,
    pop.minimum_age, 
    pop.maximum_age,
    pop.gender,
    zipcd.zipcode,
    CASE zipcd.state_code
      WHEN 'MA' THEN 'Boston'
      WHEN 'IL' THEN 'Chicago'
      WHEN 'GA' THEN 'Atlanta'
      WHEN 'PA' THEN 'Philadelphia'
  END
    city,
    zipcd.zipcode_geom
  FROM
    `bigquery-public-data.utility_us.zipcode_area` zipcd,
    `bigquery-public-data.census_bureau_usa.population_by_zip_2010` pop
  WHERE
    zipcd.state_code IN ('MA',
      'IL',
      'PA',
      'GA')
    AND ( zipcd.city LIKE '%Atlanta%'
      OR zipcd.city LIKE '%Boston%'
      OR zipcd.city LIKE '%Chicago%'
      OR zipcd.city LIKE '%Philadelphia%' )
    AND SUBSTR(CONCAT('000000', pop.zipcode),-5) = zipcd.zipcode
  GROUP BY
    pop.minimum_age, 
    pop.maximum_age,
    pop.gender,
    zipcd.zipcode,
    CASE zipcd.state_code
      WHEN 'MA' THEN 'Boston'
      WHEN 'IL' THEN 'Chicago'
      WHEN 'GA' THEN 'Atlanta'
      WHEN 'PA' THEN 'Philadelphia'
  END
    ,
    zipcd.zipcode_geom
    limit 100

,population,minimum_age,maximum_age,gender,zipcode,city,zipcode_geom
0,23129,NaN,NaN,None,30345,Atlanta,"POLYGON ((-84.315404 33.850493,-84.312023 33.8..."
1,11945,NaN,NaN,male,30345,Atlanta,"POLYGON ((-84.315404 33.850493,-84.312023 33.8..."
2,860,0.0,4.0,male,30345,Atlanta,"POLYGON ((-84.315404 33.850493,-84.312023 33.8..."
3,762,5.0,9.0,male,30345,Atlanta,"POLYGON ((-84.315404 33.850493,-84.312023 33.8..."
4,632,10.0,14.0,male,30345,Atlanta,"POLYGON ((-84.315404 33.850493,-84.312023 33.8..."
...,...,...,...,...,...,...,...
95,1,75.0,79.0,female,19113,Philadelphia,"POLYGON ((-75.294508 39.866018,-75.286723 39.8..."
96,0,80.0,84.0,female,19113,Philadelphia,"POLYGON ((-75.294508 39.866018,-75.286723 39.8..."
97,1,85.0,NaN,female,19113,Philadelphia,"POLYGON ((-75.294508 39.866018,-75.286723 39.8..."
98,11063,NaN,NaN,None,60482,Chicago,"POLYGON ((-87.810496 41.69466,-87.797857 41.69..."


The population is by age and gender. The zip code dataset provides geo information as a polygon.

Next we check which intersection coordinates are within a polygon. So we can match intersection to zip code.

In [4]:
%%bigquery df
WITH

  # population per zipcode
  # (for simplicity ignore gender and age information)

  zip_info AS(
  SELECT
    pop.minimum_age, 
    pop.maximum_age,
    pop.gender,
    SUM(pop.population) AS population,
    zipcd.zipcode,
    CASE zipcd.state_code
      WHEN 'MA' THEN 'Boston'
      WHEN 'IL' THEN 'Chicago'
      WHEN 'GA' THEN 'Atlanta'
      WHEN 'PA' THEN 'Philadelphia'
  END
    city,
    zipcd.zipcode_geom
  FROM
    `bigquery-public-data.utility_us.zipcode_area` zipcd,
    `bigquery-public-data.census_bureau_usa.population_by_zip_2010` pop
  WHERE
    zipcd.state_code IN ('MA',
      'IL',
      'PA',
      'GA')
    AND ( zipcd.city LIKE '%Atlanta%'
      OR zipcd.city LIKE '%Boston%'
      OR zipcd.city LIKE '%Chicago%'
      OR zipcd.city LIKE '%Philadelphia%' )
    AND SUBSTR(CONCAT('000000', pop.zipcode),-5) = zipcd.zipcode
  GROUP BY
    pop.minimum_age, 
    pop.maximum_age,
    pop.gender,
    zipcd.zipcode,
    CASE zipcd.state_code
      WHEN 'MA' THEN 'Boston'
      WHEN 'IL' THEN 'Chicago'
      WHEN 'GA' THEN 'Atlanta'
      WHEN 'PA' THEN 'Philadelphia'
  END
    ,
    zipcd.zipcode_geom),
  
  # spatial test and train data
  
  train_and_test AS (
  SELECT
    tr.intersectionId,
    tr.longitude,
    tr.latitude,
    tr.city
  FROM
    `kaggle-competition-datasets.geotab_intersection_congestion.train` tr
  UNION DISTINCT
  SELECT
    ts.intersectionId,
    ts.longitude,
    ts.latitude,
    ts.city
  FROM
    `kaggle-competition-datasets.geotab_intersection_congestion.test` ts),
  
  # Zipcode and Population per Intersection
  
  pop_per_intersection AS (
  SELECT
    t.intersectionId,
    zi.population,
    zi.zipcode,
    t.city,
    zi.minimum_age, 
    zi.maximum_age,
    zi.gender,
    zi.zipcode_geom
  FROM
    train_and_test t,
    zip_info zi
  WHERE
    t.city = zi.city
    AND ST_CONTAINS( ST_GEOGFROMTEXT(zi.zipcode_geom),
      ST_GeogPoint(longitude,
        latitude)))
  
# fill empty/missing zipcodes and population

SELECT
  t.city,
  t.intersectionId, 
  p.minimum_age, 
  p.maximum_age,
  p.gender,
  coalesce(p.population,
    round(AVG(p.population) OVER(PARTITION BY t.city, p.minimum_age, p.maximum_age, p.gender))) AS population,
  coalesce(p.zipcode, 'N/A') AS zipcode,
  CASE
    WHEN p.zipcode IS NULL THEN 1
  ELSE
  0
END AS zip_code_na
#--,
#--ST_GeogPoint(t.longitude,
#--        t.latitude) intersection_gp,
#p.zipcode_geom
FROM
  train_and_test t
LEFT OUTER JOIN
  pop_per_intersection p
ON
  (t.city = p.city
    AND t.intersectionId = p.intersectionId);

In [5]:
df.head()

,city,intersectionId,minimum_age,maximum_age,gender,population,zipcode,zip_code_na
0,Atlanta,300,NaN,NaN,None,49736.0,30318,0
1,Atlanta,314,NaN,NaN,None,14857.0,30354,0
2,Atlanta,231,NaN,NaN,None,49736.0,30318,0
3,Atlanta,191,NaN,NaN,None,54094.0,30331,0
4,Atlanta,479,NaN,NaN,None,14857.0,30354,0


Check number of unique intersections:

In [6]:
print('Assert, number of unique intersactions as expected:', df.groupby(['city']).intersectionId.nunique().sum()==6381)

Assert, number of unique intersactions as expected: True


# Export to csv
Missing population (where zipcode == 'N/A') is imputed with mean over gender, age and city. Imputed data is flagged in zip_code_na.

In [7]:
df.to_csv('pop_zipcode_intersec.csv', index=False)